## Code to run CorrCal with dish scatter

In [35]:
import numpy as np
import random
import h5py,time, matplotlib.pyplot as plt
from scipy.optimize import fmin_cg, minimize
from drift.core import manager
import corrcal
import sys
sys.path.insert(0,'/home/zahra/PIPELINE')
from hirax_transfer import core
import scipy as sp
from cora.util import hputil
from astropy.stats import gaussian_fwhm_to_sigma
from hirax_transfer.beams import separations
import healpy as hp
from cora.core import skysim
from cora.foreground import gaussianfg, galaxy
from cora.util import coord
from drift.core import visibility
sys.path.insert(0,'/home/zahra/hirax_tools/')
from hirax_tools import array_config
from log_red_cal_new import Visibilities_grid, Bls_counts, colour_scatterplot, Scatterplot, index_find
from cora.foreground import poisson as ps

Prod params file used to generate x and y antenna positions, assuming single feed for each antenna


In [36]:
m = manager.ProductManager.from_config('/home/zahra/PIPELINE/8by8_no_sims/prod_params_custom.yaml') 

t = m.telescope
Nfeeds,_= t.feedpositions.shape
Ndish = np.int(Nfeeds/2)
Nbls = np.int(Ndish*(Ndish-1)/2)

x = t.feedpositions[:,0][:Ndish] 
y = t.feedpositions[:,1][:Ndish]


Product directory: /home/zahra/PIPELINE/8by8_no_sims/bt_matrices


In [37]:
x = x + np.load('/home/zahra/corrcal2/random_pt1_x_64.npy')[:Ndish]
y = y + np.load('/home/zahra/corrcal2/random_pt1_y_64.npy')[:Ndish]

I get the redundant blocks using a separate code

In [38]:
bl_arr_dish_indices, sum_counts, bl_redundancy_ordered, _, _ = Bls_counts(m)
lims = np.append(0, np.cumsum(bl_redundancy_ordered)) #these are the edges of the redundant 
#blocks

In [39]:
ant1 = bl_arr_dish_indices[:,0].astype(int)
ant2 = bl_arr_dish_indices[:,1].astype(int)

In [40]:
Frequency = 400. #MHz
Nside = 256
zenith = t.zenith

In [41]:
def baseline_arr(freq):
    '''
    Calculates baselines in uv coordinates
    
    Parameters
    ----------
    freq : float
        The frequency at which you are observing (in MHz).

    Returns
    -------
    baselines : ndarray
        The baselines in uv coordinates (Nbls,2)
        Organised according to baseline redundancy
    '''
    wavelength = (3.e8)/(freq*10.**6)
    bl_arr_uv = np.zeros((Nbls,2))
    for i in range(len(bl_arr_dish_indices)):
        dish_0, dish_1 = bl_arr_dish_indices[i]
        bl_ind = [np.int(dish_0), np.int(dish_1)]
        u_coord = (x[bl_ind[1]] - x[bl_ind[0]])/wavelength
        v_coord = (y[bl_ind[1]] - y[bl_ind[0]])/wavelength
        bl_arr_uv[i,:] += np.array([u_coord, v_coord])
    return bl_arr_uv

In [42]:
def Gauss_beam(freq, nside): 
    '''
    Calculates the beam model
    We assume a Gaussian beam, and assume the same beam for each antenna
    
    Parameters
    ----------
    freq : float
        The frequency at which you are observing (in MHz).
    nside : int
        The dimensions of the HEALPix map used
        
    Returns
    -------
    Beam : ndarray
        Beam model for each pixel (npix,)
    '''
    wavelength = (3.e8)/(freq*10.**6)    
    fwhm = 1.* wavelength/6.
    sigma_beam = gaussian_fwhm_to_sigma*fwhm
    angpos = hputil.ang_positions(nside)
    seps = separations(angpos, zenith)
    beam = np.exp(-seps**2/2/sigma_beam**2)
    return beam

The Mu function follows the equation $\mu_n = \int S^n \frac{dN}{dS} dS$, with n=2 used for visibilities and n=3 for sky cov

In [43]:
def Mu(index):
    '''
    Calculates mu by assuming a differential source count, dN/dS, as per Santos et al, 2005
    
    Parameters
    ----------
    index : int
        Input the index of source flux, S, depending on what you which to calculate
        n = 2 for visibilities due to unresolved point sources following a Poisson distribution
        n = 3 for the covariance of the visibilities due to these point sources

    Returns
    -------
    mu_n : float
        A constant, computed as per the equation above
    '''
    S_max =  5 * 12 * 1.e-6 * np.sqrt(1024./Ndish) #12 micro jansky sensitivity for full array,
    # scaled for our size array, multiplied by 5 for a 5\sigma detection
    alpha = 4000 
    beta = 1.75
    mu_index = alpha/(index-beta) * (S_max**(index-beta))
    return mu_index 

I give details of generating the bright point source map (These are taken from cora - one of the Radio Cosmology packages). The functions below show that fluxes from a flux distribution are placed in random pixels, such that we can treat individual pixels as sources.

Please note that the next three cells are taken from cora and are not written by me

In [44]:
flux_min = 2.
flux_max = 2.75

In [45]:
gamma1 = 1.75
gamma2 = 2.51
S_0 = 0.88
k1 = 1.52e3

spectral_mean = -0.7
spectral_width = 0.1

spectral_pivot = 151.0

def source_count(flux):
    r"""Power law luminosity function."""

    s = flux / S_0

    return k1 / (s**gamma1 + s**gamma2)

def spectral_realisation(flux, freq):
    r"""Power-law spectral function with Gaussian distributed index."""

    ind = spectral_mean + spectral_width * np.random.standard_normal(flux.shape)

    return flux * (freq / spectral_pivot)**ind


In [46]:
def generate_population(area):
    r"""Create a set of point sources.

    Parameters
    ----------
    area : float
        The area the population is contained within (in sq degrees).

    Returns
    -------
    sources : ndarray
        The fluxes of the sources in the population.
    """

    rate = lambda s: flux_min*np.exp(s)*area*source_count(flux_min*np.exp(s))
    fluxes = flux_min * np.exp(ps.inhomogeneous_process_approx(np.log(flux_max/flux_min), rate))

    return fluxes


def getsky(freq, nside):
    """Simulate a map of point sources.

    Returns
    -------
    sky : ndarray [nfreq, npix]
        Map of the brightness temperature on the sky (in Jy).
    """

    npix = 12*nside**2

    nfreq =1

    sky = np.zeros((nfreq, npix), dtype=np.float64)

    fluxes = generate_population(4*np.pi)

    sr = spectral_realisation(fluxes[:,np.newaxis], freq)

    for i in range(sr.shape[0]):
        # Pick random pixel
        ix = int(np.random.rand() * npix)

        sky[:, ix] += sr[i,:]
    return sky


In [47]:
Map = getsky(Frequency, Nside)[0] # I save this array for each flux range that I consider so 
#that the source positions are the same for each run

Below is the function to generate the source vector. Ideally, when computing the per visibility response to each source for each baseline, I would consider each pixel in the HEALPix map. This takes about two hours to finish running, and scales up considerably with an increase in the number of bright sources. Thus, I only include the pixels that contribute to the visilities (i.e. include pixels that give non-zero visibilities). As a result, I multiply the map values and beammodel, and consider the pixels for which this product is larger than some value (set to $10^{-3}$ in the function below)

In [59]:
def Src_vector(freq, nside, map):
    '''
    Calculates the source vector, S, that is input into CorrCal (for known source positions), 
    and contains the per visibility response to bright sources with known position. 
    Vector S is then used to calculate the total visibility contribution from bright sources. 
    Note that we have separated S into its real and imaginary but have not separated the 
    visibilities into these components as yet
    
    Parameters
    ----------
    freq : float
        The frequency at which you are observing (in MHz).
    nside : int
        The dimensions of the HEALPix map used
    map : ndarray
        The map of bright point sources that you are using (npix,)
        
    Returns
    -------
    source vector : tuple
        Vis_bright_sources : Visibilities due to bright point sources (Nbls,)
        src_total : Per visibility response to sources with known positions (2*Nsources, 2*Nbls)
    '''
    fringe = visibility.fringe
    angpos = hputil.ang_positions(nside)
    beammodel = Gauss_beam(freq, nside)
    bl_arr = baseline_arr(freq)
    map_vals = map
    vals_beam = map_vals*beammodel
    pix_valsbeam = np.where(vals_beam >1e-3)[0] #reducing this value from 1e-3 gives bad results
    # for the 1-1.2 Jy case with known positions
    source_number = len(pix_valsbeam)
    src = np.zeros((len(bl_arr_dish_indices), source_number), dtype='complex')
    for n in range(source_number):
        non_zero_pix = pix_valsbeam[n]
        for i in range(len(bl_arr_dish_indices)):
            fringe_ind = fringe(angpos[non_zero_pix], zenith, bl_arr[i])
            vis_ind_source_ind = map_vals[non_zero_pix] * beammodel[non_zero_pix]**2 * fringe_ind
            src[i, n] = vis_ind_source_ind
    Vis_bright_sources = np.sum(src, axis=1)
    src_total = np.zeros((2*Nbls, 2*source_number))
    src_total[::2,::2] = src.real
    src_total[1::2,1::2] = src.imag
    src_total = src_total.T
    return Vis_bright_sources, src_total

The visibilities due bright sources is then computed by summing over the per visibility response of each bright source, in the source vector, S, as shown below

In [60]:
Vis_bright_sources, src_total = Src_vector(Frequency, Nside, Map)

Again, to speed up the code, I do not include each pixel of the HEALPix map when generating visibilities of unresolved, dim sources, with a Poisson distribution. I only include pixels where the beam is larger than $10^{-10}$, as demonstrated in the function below.

In [ ]:
def Visibilities_poisson(freq, nside):
    '''
    Calculates visibilities due to unresolved point sources below the S_{max} set by the 
    telescope sensitivity. We assume sources follow a Poisson distribution, and do not account 
    for source clustering. 
    
    Parameters
    ----------
    freq : float
        The frequency at which you are observing (in MHz).
    nside : int
        The dimensions of the HEALPix map used
        
    Returns
    -------
    vis_poisson : ndarray
        Visibilities of unresolved sources (Nbls,)
    '''
    fringe = visibility.fringe
    angpos = hputil.ang_positions(nside)
    npix = hp.nside2npix(nside)
    beammodel = Gauss_beam(freq, nside)
    bl_arr = baseline_arr(freq)
    point_source_vis = Mu(2)
    pix_beam = np.where(beammodel>1e-10)[0]
    vis_poisson = np.array([])
    for i in range(len(bl_arr_dish_indices)):
        fringe_ind = fringe(angpos[pix_beam], zenith, bl_arr[i])
        vis_ind = 4 * np.pi/npix * point_source_vis * np.sum(beammodel[pix_beam]**2 * fringe_ind)
        vis_poisson = np.append(vis_poisson, vis_ind)
    return vis_poisson

In [ ]:
Vis_poisson = Visibilities_poisson(Frequency, Nside)

In [ ]:
Vis_total = Vis_poisson + Vis_bright_sources

The redundant blocking that I use in the function below is the same as that used for the perfectly redundant case. 

In [ ]:
def Covariance_poisson(freq, nside):
    '''
    Covariance matrix computed using unresolved point sources following a Poisson distribution,
    and we compute the covariance at a single frequency, across different baselines. 
    Covariance matrices are computed separately for each redundant block such that we do not 
    compute a full (Nbls, Nbls) matrix
    
    Parameters
    ----------
    freq : float
        The frequency at which you are observing (in MHz).
    nside : int
        The dimensions of the HEALPix map used
        
    Returns
    -------
    vis_poisson : dict
        Separate n x n covariance matrices for each redundant block, with the matrix size 
        depending on the baseline redundancy
    '''
    angpos = hputil.ang_positions(nside)
    npix = hp.nside2npix(nside)
    point_source_cov = Mu(3)
    bl_arr = baseline_arr(freq)
    beammodel = Gauss_beam(freq, nside)
    pix_beam = np.where(beammodel>1e-10)[0]

    Cov_dic ={} 
    for ubl_k in range(len(lims)-1):
        block_k = bl_redundancy_ordered[ubl_k]
        cov_k =np.zeros((block_k,block_k), dtype='complex') 
        for bl_w in range(block_k):
            for bl_z in range(block_k):
                u_alph_bet = bl_arr[bl_w] - bl_arr[bl_z]
                fringes_ind = visibility.fringe(angpos[pix_beam], zenith, u_alph_bet)
                cov_k[bl_w][bl_z] = 4 * np.pi/npix * point_source_cov * np.sum(beammodel[pix_beam]**4*fringes_ind)
        Cov_dic[ubl_k]= cov_k
    return Cov_dic

In [ ]:
def Vecs(freq, nside):
    '''
    Decomposes the covariance matrix into eigenvalues and eigenvectors, separately for each
    redundant block. This is the R vector that is input into CorrCal (if you are not using the 
    redundant case in CorrCal). We set a threshold for the eigenvalues that are considered,
    which determines the number of vectors, nvec. Also R is separated into its real and 
    imaginary components
    
    Parameters
    ----------
    freq : float
        The frequency at which you are observing (in MHz).
    nside : int
        The dimensions of the HEALPix map used
        
    Returns
    -------
    vecs : ndarray
        R vector (2*Nvec, 2*Nbls)
    '''
    Cov_dic = Covariance_poisson(freq, nside)
    thresh = 1.e-6

    ind_array=np.array([])
    for i in range(len(lims)-1):
        myeig, myvecs = np.linalg.eig(Cov_dic[i])    
        ind = np.sum(myeig > thresh * myeig.max()) #Only eigenvalues with 10^{-6} times the max
        # eigenvalue are considered
        ind_array = np.append(ind_array, ind)

    nvec = np.int((ind_array).max())

    vecs = np.zeros((2*Nbls,2*nvec))

    for i in range(len(lims)-1):
        myeig, myvecs = np.linalg.eig(Cov_dic[i])
        ind = myeig >thresh * myeig.max() # picking up an index
        myeig = myeig[ind] # pick max eigenvalue
        myvecs = myvecs[:,ind]

        for j in range(len(myeig)): 
                myvecs[:,j]= myvecs[:,j]*np.sqrt(myeig[j])
                vecs[2*lims[i]:2*lims[i+1]:2, 2*j] = np.column_stack(myvecs[:,j].real)
                vecs[(2*lims[i]+1):(2*lims[i+1]+1):2, 2*j+1] = np.column_stack(myvecs[:,j].imag)
    #Note that every first row and first column contains real components, 
    #every second row and column contains imaginary components
    vecs= vecs.T
    return vecs

In [ ]:
vecs = Vecs(Frequency, Nside)

True gains which we set to 1, so that the real and imaginary components correspond to the
amplitude and phase, respectively

In [ ]:
gain = np.ones(Ndish)
sim_gains = np.ones(2*Ndish)
sim_gains[::2] = gain.real
sim_gains[1::2] = gain.imag

In [ ]:
src_zeros = np.zeros(2*Nbls) # used if the source positions are unknown

In [ ]:
v1=np.zeros(2*Nbls)
v1[0::2]=1
v2=np.zeros(2*Nbls)
v2[1::2]=1
vecs_redundant = np.vstack([v1,v2])*1.e3 # used for the redundant case in CorrCal

In [ ]:
def fit_gains(freq, nside, src_array, vecs, visibilities):
    '''
    Computes the recovered gains for both amplitude and phase. The inputs include the source 
    vector, S. The two options for the vector depend on whether or not source positions are 
    known.
    The vector, R, constructed from the visibility covariance, and the visibilities. The two 
    options for the vector depend on whether or not you are using the redundant case.
    The visibilities input here are noiseless. Noise is input into the visibilities in this 
    function, and visibilities are then separated into real and imaginary components. I either 
    use the sum of unresolved and bright sources, or just the unresolved sources.
    
    Parameters
    ----------
    freq : float
        The frequency at which you are observing (in MHz).
    nside : int
        The dimensions of the HEALPix map used
    src_array : ndarray
        The source vector, S (2*Nsources, 2*Nbls)
    vecs : ndarray
        The covariance vector, R (2*Nvec, 2*Nbls)
    visibilities : ndarray
        Noiseless visibilities (Nbls,) 
        
    Returns
    -------
    recovered gains : ndarray
        Recovered gains separated in real (amplitude) and imaginary (phase) components (2*Ndish)
    '''
    S_max =  5 * 12 * 1.e-6 * np.sqrt(1024./Ndish)
    sigma = S_max*0.01
    diag = sigma**2*np.ones(2*Nbls)
    runs = 500
    rand_normal = np.random.normal
    random_gain = rand_normal(0,1.e-3, 2*Ndish) #initial guess input into corrcal
    gvec = np.array([])
    for i in range(len(sim_gains)):
        gvec=np.append(gvec,sim_gains[i]+random_gain[i])

    get_chisq = corrcal.get_chisq
    get_gradient = corrcal.get_gradient
    mat = corrcal.Sparse2Level(diag,vecs,src_array,2*lims) 

    rec_gains = np.zeros((runs,Ndish*2))
    for ind_run in range(runs):
        print (ind_run)
        N_real = rand_normal(0, sigma, Nbls)
        N_imag = rand_normal(0, sigma, Nbls)
        N_comp = np.array([])
        for i in range(len(N_real)):
            N_comp = np.append(N_comp,complex(N_real[i],N_imag[i]))
        vis = visibilities + N_comp

        data = np.zeros(2*vis.size)
        data[0::2] = vis.real
        data[1::2] = vis.imag
        fac=1.;
        normfac=1.
        results = fmin_cg(get_chisq, gvec*fac, get_gradient,(data,mat,ant1,ant2,fac,normfac))
        fit_gains_run = results/fac
        rec_gains[ind_run,:] = fit_gains_run
        print (fit_gains_run[::2])
    return rec_gains

Below I list the different possibilities I consider:
- Recovered gains with bright sources in visibilities and with known positions
- Recovered gains with bright sources in visibilities and no known positions
- Recovered gains with no bright sources in visibilities (just unresolved, Poisson sources)
- Recovered gains with bright sources in visibilities, for the redundant case - no known positions and also no useful information in the R matrix 

In [ ]:
rec_gains = fit_gains(Frequency, Nside, src_total, vecs, Vis_total)
#rec_gains = fit_gains(Frequency, Nside, src_zeros, vecs, Vis_total)
#rec_gains = fit_gains(Frequency, Nside, src_zeros, vecs, Vis_poisson[1])
#rec_gains = fit_gains(Frequency, Nside, src_zeros, vecs_redundant, Vis_total)

In [1]:
def hist_rel_err_mean_std(rec_gains_amp_or_phase, sim_gains_amp_or_phase):
    '''
    Histogram of mean and standard deviation of the amplitude/phase relative error
    for a number of noise realisations. The error is computed by taking the recovered gains and
    subtracting out the true gains.
    
    Parameters
    ----------
   rec_gains_amp_or_phase  : ndarray
       The recovered gains for either the amplitude or phase
    sim_gains_amp_or_phase : ndarray
       The corresponding true gains for either the amplitude or phase
        
    Returns
    -------
    Standard deviation and mean : tuple
        Standard deviation of relative errors (Nruns,)
        Mean of relative errors (Nruns,)
    '''
    rel_error = np.abs(rec_gains_amp_or_phase - sim_gains_amp_or_phase)
    rec_gains_mean = np.mean(rel_error, axis=1) #shape is number of runs
    rec_gains_std = np.std(rel_error, axis=1, ddof=1)
    return rec_gains_std, rec_gains_mean

In [ ]:
std, mean = hist_rel_err_mean_std(rec_gains[:,::2], sim_gains[::2]) # To consider just the 
#amplitude, we take every second value

In [ ]:
plt.hist(mean,'auto') 
plt.xlabel('Mean')
#plt.xscale('log')
plt.show()

In [ ]:
np.median(mean)

In [ ]:
plt.hist(std,'auto') 
plt.xlabel('Standard deviation')
#plt.xscale('log')
plt.show()

In [ ]:
np.median(std)